#  Unit 2.4a Using Programs with Data, SQLAlchemy (NB + Notes + My Project Application + Hacks)
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
        - This Flask app object app object containss the routs and blueprints that reside to the HTTP server in the Flask
    2. SQLAlchemy db object
        - THe method where the SQLite database interaccts with an ergonomic ORM (Object Relational Mapping (ORM) is a technique used in creating a "bridge" between object-oriented programs and, in most cases, relational database) in the SQAlchemy db object.

In [1]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

### Notes
- Setting up variables
- Setting up DB

## APP and DB are objects we have been using all year
- APP and the db Alchemy, this data is getting assinged into this object
- Flask is a template and we havent assigned anything to it yet.

## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
        - Class user is defining the template for user. It is a class definition template. WE use this template to create objects like user.
        - We need a user template because we have different users, but we shouldn't start from scratch for each one. 
    - db.Model inheritance
        - Defining a class is called inheritance. For class user we are inheriting the ability for out class to use databast properties, methods, and attributes. The code is defined somewhere else but we are going to inherit all the properties.
        - Since we inherited these methods, we are able to use them to add to our template.
    - _init_ method
        - Usually called "constructor" and means that the init method is used to help us instantiate an object from our class user template.
        - With the init method we look at U1 and we bas username, ID, etc. The init method will set the properties as attributes are parameters that instantuate a U1 object. 
    - ```@property```, ```@<column>.setter```
        - We want to see what is inside as well as change the attribute once we created them (setters and getters).
            - setters let us change the attriubtes, getters allow us to retrive data out of the object.
    - create, read, update, delete methods
        - typicly when you create a class you use these methods in order to make it easier to interact with the properties of the database.

    - This next code is a definition template, so we wont get a response out of it
        - it is a point of reference as of now, becuase we have not referenced or instantuatied it yet.

In [2]:
class etrack_user(db.Model):
    __tablename__ = 'etrack_users'  # table name is plural, class name is singular
    # Define the User schema with "vars" from object
    _date = db.Column(db.String(255), primary_key=True)
    _savedWorkouts = db.Column(db.PickleType, unique=False, nullable=False)

    def __init__(self, date, savedWorkouts):
        self._date = date
        self._savedWorkouts = savedWorkouts

    @property
    def date(self):
        return self._date
    
    @date.setter
    def date(self, date):
        self._date = date

    @property
    def savedWorkouts(self):
        return self._savedWorkouts

    @savedWorkouts.setter
    def savedWorkouts(self, savedWorkouts):
        self._savedWorkouts = savedWorkouts

    @property
    def dictionary(self):
        dict = {
            "date" : self.date,
            "saved workouts" : self.savedWorkouts,
        }
        return dict

    def __str__(self):
        return json.dumps(self.dictionary)
    
    def __repr__(self):
        return f'etrack_user(date={self._date}, savedWorkouts={self._savedWorkouts})'


    # CRUD stuff
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "date": self.date,
            "savedWorkouts": self.savedWorkouts,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, date, savedWorkouts):
        """only updates values with length"""
        if len(date) > 0:
            self.date = date
        # if len(savedWorkouts) > 0:
        self.savedWorkouts = savedWorkouts
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
        - The db.create_all() method on the db object creates all the relevant SQL tables based on the defined models.
    2. User Object Constructors
        - Instantaniate the instance of the user class with the specific parameters
    3. Try / Except 
        - Finds python exceptions that could possibly be found and print an error messaage if any of these are caught.


In [3]:
def initEtrackUsers():
    with app.app_context():
        # db.init_app(app)
        db.create_all()
        r1 = etrack_user(date="25 February 2023" , savedWorkouts = ["This is a test of the new table"])

        rows = [r1]

        """Builds sample user/note(s) data"""
        for row in rows:
            try:
                row.create()
            except IntegrityError:
                '''fails with bad or duplicate data'''
                db.session.remove()
                print(f"Records exist, duplicate username, or error: {row.date}")
initEtrackUsers()

NameError: name 'IntegrityError' is not defined

### Notes
- Creates data into new table

## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password
- 
    - Enables us to do query methdods on the data that is in our class
        - IN this case we are doing it for the user ID in the database and return only the first match.
        
- Comment on purpose of following
    1. User.query.filter_by
        - Filter by a certain table column and return all relevant rows
    2. user.password
        - Retrieve the password column entry for a specific row from the database

In [4]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_date(date):
    with app.app_context():
        user = etrack_user.query.filter_by(_date=date).first()
    return user # returns user object

# Check credentials by finding user and verify password
# def check_credentials(uid, password):
#     # query email and return user record
#     user = find_by_uid(uid)
#     if user == None:
#         return False
#     if (user.is_password(password)):
#         return True
#     return False
        
#check_credentials("indi", "123qwerty")

### Notes
- Function for searching for a row in the DB

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
        - find_by_uid returns the user row corresponding to a unique ID and returns None if the user does not exist. See above for try/except.
    2. user = User(...)
        - The user object constructors (found in code)
    3. user.dob and try/except
        - Get the dob column entry for each respective row entry
    4. user.create() and try/except
        - Creates rows in the database with the daata that the current intance of the user created.

In [5]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    date = input("Enter your date")
    user = find_by_date(date)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    savedWorkouts = input("Enter your workout")
    
    # Initialize User object before date
    user = etrack_user(date=date, savedWorkouts=savedWorkouts)
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'date': '26 March 2023', 'savedWorkouts': '225lb Bench'}


### Notes
- Method for creating a row

## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
        - Returns the rows in the corresponding user class table
    2. json_ready assignment, google List Comprehension
        - Reads the user interies in the table and storesthem in a list of python objects.

In [27]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = etrack_user.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'date': '25 February 2023',
  'savedWorkouts': ['This is a test of the new table']},
 {'date': '25 March 2023', 'savedWorkouts': ['135 lb Bench']}]

### Notes
- Method for returning all rows

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

In [24]:
def update():
    date = input("Enter your date")
    row = find_by_date(date)
    try:
        print("Found\n", row.read())
        pass
    except:
        return("User wasn't found")
    
    # request value that ensure creating valid object
    workout = input("Enter your workout")
           
    # write object to database
    with app.app_context():
        try:
            object = row.update(date, workout)
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
update()

Found
 {'date': '12 March 2023', 'savedWorkouts': '2 Mile Run'}
Created
 {'date': '12 March 2023', 'savedWorkouts': '3 Mile Run'}


In [26]:
def delete():
    date = input("Enter your date")
    row = find_by_date(date)
    try:
        print("Found\n", row.read())
        pass
    except:
        return("User wasn't found")
    
    # request value that ensure creating valid object
           
    # write object to database
    with app.app_context():
        try:
            row.delete()
            print("Deleted\n")
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
delete()

Found
 {'date': '12 March 2023', 'savedWorkouts': '2 Mile Run'}
Deleted



In [21]:
import sqlite3

database = 'instance/sqlite.db' # this is location of database

def schema():
    
    # Connect to the database file
    conn = sqlite3.connect(database)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Fetch results of Schema
    results = cursor.execute("PRAGMA table_info('etrack_users')").fetchall()

    # Print the results
    for row in results:
        print(row)

    # Close the database connection
    conn.close()
    
schema()

(0, '_date', 'VARCHAR(255)', 1, None, 1)
(1, '_savedWorkouts', 'BLOB', 1, None, 0)


In [17]:
# Menu, to run other cells from one control point
def menu():
    operation = input("Enter: (C)reate (R)ead (U)pdate or (D)elete or (S)chema")
    if operation.lower() == 'c':
        create()
    elif operation.lower() == 'r':
        read()
    elif operation.lower() == 'u':
        update()
    elif operation.lower() == 'd':
        delete()
    elif operation.lower() == 's':
        schema()
    elif len(operation)==0: # Escape Key
        return
    else:
        print("Please enter c, r, u, or d") 
    menu() # recursion, repeat menu
        
try:
    menu() # start menu
except:
    print("Perform Jupyter 'Run All' prior to starting menu")

Created
 {'date': '16 March 2023', 'savedWorkouts': ['145 lb Overhead']}
Found
 {'date': '16 March 2023', 'savedWorkouts': ['145 lb Overhead']}
Created
 {'date': '16 March 2023', 'savedWorkouts': '150 lb Overhead'}
Found
 {'date': '16 March 2023', 'savedWorkouts': ['145 lb Overhead']}
Deleted

Perform Jupyter 'Run All' prior to starting menu
